# TODO:
- Check which crimes are correl. with socio-economic status (proxy: race (and sex and age?))
- Check whether race/age/sex is correl. with COMPAS scores
- Check whether race/age/sex is correl. with OUR scores
- Try out some other models as well (SVMs, Random Forests, Naive Bayes) 

# Trying stuff out
Import our tools

In [1]:
import pandas as pd

Load in the relevant data, parsing the date fields to a format pandas understands on the way.

In [2]:
df = pd.read_csv(
    'compas-scores.csv',
    usecols=[
        'sex',
        'age',
        #'age_cat',
        'race',
        'juv_fel_count',
        #'decile_score',
        'juv_misd_count',
        'juv_other_count', 
        'priors_count',
        'days_b_screening_arrest',
        'c_jail_in',
        'c_jail_out',
        'c_offense_date',
        'c_arrest_date',
        'c_days_from_compas', 
        'c_charge_degree',
        'c_charge_desc',
        'is_recid',
        #'num_r_cases',
        #'r_charge_degree', 
        #'r_days_from_arrest', 
        #'r_offense_date',
        #'r_charge_desc', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'is_violent_recid',
        #'num_vr_cases', 
        #'vr_charge_degree', 
        #'vr_offense_date',
        #'vr_charge_desc', 
        #'v_decile_score',
        #'v_score_text', 
        #'v_screening_date', 
        #'score_text', 
    ],
    parse_dates=[
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ]
)

In [3]:
df = pd.read_csv(
    'compas-scores.csv',
    usecols=[
        'sex',
        'age',
        #'age_cat',
        'race',
        'juv_fel_count', 
        #'decile_score',
        'juv_misd_count',
        'juv_other_count', 
        'priors_count',
        'days_b_screening_arrest',
        'c_jail_in',
        'c_jail_out',
        'c_offense_date',
        'c_arrest_date', 
        'c_days_from_compas', 
        'c_charge_degree',
        'c_charge_desc', 
        'is_recid',
        #'num_r_cases',
        #'r_charge_degree', 
        #'r_days_from_arrest', 
        #'r_offense_date',
        #'r_charge_desc', 
        #'r_jail_in', 
        #'r_jail_out', 
        'is_violent_recid',
        #'num_vr_cases', 
        #'vr_charge_degree', 
        #'vr_offense_date',
        #'vr_charge_desc', 
        #'v_decile_score',
        #'v_score_text', 
        #'v_screening_date', 
        #'score_text', 
    ],
    parse_dates=[
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ]
)

In [4]:
print(df.shape)
df = df[df['c_days_from_compas'] <= 10]
print(df.shape)
print(df.columns)
df['c_charge_desc'] = df.c_charge_desc.apply(lambda x: str(x).lower())
print(df.c_charge_desc[['agg' in desc for desc in df.c_charge_desc.tolist()]].unique())

(11757, 17)
(8938, 17)
Index(['sex', 'age', 'race', 'juv_fel_count', 'juv_misd_count',
       'juv_other_count', 'priors_count', 'days_b_screening_arrest',
       'c_jail_in', 'c_jail_out', 'c_offense_date', 'c_arrest_date',
       'c_days_from_compas', 'c_charge_degree', 'c_charge_desc', 'is_recid',
       'is_violent_recid'],
      dtype='object')
['aggravated assault w/firearm' 'agg fleeing and eluding'
 'agg battery grt/bod/harm' 'aggravated battery / pregnant'
 'aggravated assault w/dead weap' 'aggrav battery w/deadly weapon'
 'aggrav stalking after injunctn'
 'aggravated battery (firearm/actual possession)'
 'agg fleeing/eluding high speed' 'aggravated battery'
 'aggrav child abuse-agg battery' 'aggravated assault'
 'agg assault w/int com fel dome' 'aggrav child abuse-causes harm'
 'stalking (aggravated)' 'aggravated assault w/o firearm'
 'aggr child abuse-torture,punish' 'aggravated battery on 65/older'
 'aggravated battery (firearm)' 'aggravated manslaughter child'
 'agg abuse 

Prune non-violent out from the dataset

In [5]:
possibly_violent = ('assault')
violent_kw = {'aggravated', 'battery', 'burglary', 'agg', 'violence', 'manslaughter', 'murder', 'bodily',
                    'armed', 'cruelty', 'assault'}
non_violent_kw = {'w/o violence', 'without violence', 'no bodily harm'}

def violent_charge(x):
    s = set(x.split())
    if s - non_violent_kw != s: return False
    if s - violent_kw != s: return True
    else: return False
    
df = df[df.c_charge_desc.apply(violent_charge)]
print(df.c_charge_desc)

0           aggravated assault w/firearm
2         felony battery w/prior convict
6                                battery
7              possession burglary tools
9                                battery
14                               battery
15                               battery
16                               battery
20                               battery
25         battery on law enforc officer
29        lewd/lasc battery pers 12+/<16
32                               battery
40        felony battery w/prior convict
42                               battery
47       burglary structure assault/batt
49               agg fleeing and eluding
51                               battery
52                               battery
58        viol injunct domestic violence
59          burglary unoccupied dwelling
62            burglary dwelling occupied
64                               battery
65                               battery
78          attempt armed burglary dwell
83              

In [6]:
df['c_charge_desc'].value_counts().to_csv('unique_c_charge_desc_lowercase.csv')

Transform all string data to lowercase and remove extra whitespace. This handles some problematic cases where we have e.g. 'Id theft' and 'ID Theft' as separate labels.

In [7]:
df = df.applymap(lambda s : s.lower().strip() if type(s) == str else s)
df['c_charge_desc'].value_counts().to_csv('cadasdas.csv')

Transform all string columns to groups of binary columns. This is probably not the smartest way to go about this, given that we have so many crime labels. The smart thing to do would probably be some type of a crime classification system, e.g. is_violent, is_with_weapon, etc. But that would require a lot of manual work.

In [8]:
df = pd.get_dummies(df)

Check if we have any more object columns (should be none)

In [9]:
df.dtypes.value_counts()

uint8             79
int64              7
datetime64[ns]     4
float64            2
dtype: int64

Calculate jail stay lengths

In [10]:
df['c_days_in_jail'] = (df['c_jail_out'] - df['c_jail_in']).dt.days.fillna(0).astype(int)
#df['r_days_in_jail'] = (df['r_jail_out'] - df['r_jail_in']).dt.days.fillna(0).astype(int)

Drop date columns

In [11]:
df.drop([
        'c_jail_in', 
        'c_jail_out', 
        'c_offense_date', 
        'c_arrest_date', 
        #'r_offense_date', 
        #'r_jail_in', 
        #'r_jail_out', 
        #'vr_offense_date',
        #'v_screening_date'
    ],
    axis='columns',
    inplace=True
)

Check dtypes again, should be no dates remaining

In [12]:
df.dtypes.value_counts()

uint8      79
int64       8
float64     2
dtype: int64

Still a few floats, see what that is all about

In [13]:
df.select_dtypes(include=['float64'])

,days_b_screening_arrest,c_days_from_compas
0,-1.0,1.0
2,-1.0,1.0
6,0.0,0.0
7,-1.0,1.0
9,-1.0,1.0
14,0.0,0.0
15,0.0,0.0
16,0.0,0.0
20,0.0,0.0
25,-1.0,1.0


At least two look like only NaNs, check to be sure

In [14]:
#print(df['num_vr_cases'].value_counts())
#print(df['num_r_cases'].value_counts())

Yeah, NaNs only. We can drop those. The others look pretty meaningless as well, so drop those too.

In [15]:
df.drop([
        #'num_r_cases',
        #'num_vr_cases',
        'days_b_screening_arrest',
        'c_days_from_compas',
        #'r_days_from_arrest'

    ],
    axis='columns',
    inplace=True
)

Check if we have any more NaNs hanging around

In [16]:
for col in df:
    count = len(df[col]) - df[col].count()
    if count:
        print(col, count)

Should be no more NaNs to go, so we can continue to learning.

Split to target and explanatory variables

In [17]:
print(df.columns)
predicted_variable = 'is_violent_recid'
df_ = df.query(predicted_variable + ' in (0, 1)')
print('negative samples: ', len(df) - len(df_))
X = df_.loc[:, df_.columns != predicted_variable]
y = df_.loc[:, df_.columns == predicted_variable]
print(X.shape, y.shape)
n_pos = y[predicted_variable].sum()
print(n_pos / (y.shape[0]))

Index(['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count',
       'priors_count', 'is_recid', 'is_violent_recid', 'sex_female',
       'sex_male', 'race_african-american', 'race_asian', 'race_caucasian',
       'race_hispanic', 'race_native american', 'race_other',
       'c_charge_degree_f', 'c_charge_degree_m',
       'c_charge_desc_agg abuse elderlly/disabled adult',
       'c_charge_desc_agg assault w/int com fel dome',
       'c_charge_desc_agg battery cause bodily',
       'c_charge_desc_agg battery grt/bod/harm',
       'c_charge_desc_agg fleeing and eluding',
       'c_charge_desc_agg fleeing/eluding high speed',
       'c_charge_desc_aggrav battery w/deadly weapon',
       'c_charge_desc_aggrav child abuse-agg battery',
       'c_charge_desc_aggravated animal cruelty',
       'c_charge_desc_aggravated assault',
       'c_charge_desc_aggravated assault w/dead weap',
       'c_charge_desc_aggravated assault w/firearm',
       'c_charge_desc_aggravated assault w/o firear

Split to train and test sets

In [22]:
from sklearn import model_selection

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1, random_state=12)
pos = y_train.sum()
print(pos, len(y_train) - pos)
pos = y_test.sum()
print(pos, len(y_test) - pos)

is_violent_recid    289
dtype: int64 is_violent_recid    2528
dtype: int64
is_violent_recid    24
dtype: int64 is_violent_recid    290
dtype: int64


Train a ridge regression model and check the $R^2$ score. Score of 1 is best possible, values can range to $-\inf$. RidgeCV automatically cross validates (leave-one-out) to figure out the best hyperparameter $\alpha$ 

In [58]:
from sklearn import linear_model, ensemble, neural_network
from sklearn.metrics import precision_recall_fscore_support as pr_f1, accuracy_score as acc, matthews_corrcoef
from time import time
import numpy as np


def print_score(modelname, y, y_hat):
    accuracy = acc(y, y_hat)
    matt = matthews_corrcoef(y.values.ravel(), y_hat)
    p, r, f1, sup = pr_f1(y, y_hat)
    print(modelname)
    print('accuracy:', acc(y, y_hat))
    print('precision:', tuple(p))
    print('recall:', tuple(r))
    print('f1:', tuple(f1))
    print('Matthew:', matt)
    print('')
    
naive_y = np.zeros(len(y_test))
print_score('Naive', y_test, naive_y)

model = linear_model.LogisticRegression()
model.fit(X_train, y_train.values.ravel())
print_score('Logreg', y_test, model.predict(X_test))

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train.values.ravel())
print_score('KNN', y_test, knn.predict(X_test))

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train.values.ravel())
print_score('RF', y_test, rf.predict(X_test))

start = time()
from sklearn.ensemble import GradientBoostingClassifier
gbt = GradientBoostingClassifier(learning_rate=1e-2, n_estimators=2000, subsample=.8)
gbt.fit(X_train, y_train.values.ravel())
print_score('GBTree', y_test, gbt.predict(X_test))
print('gbt trained in {:.2f} seconds'.format(time() - start))

start = time()
model = neural_network.MLPClassifier(hidden_layer_sizes=((200,)*2), solver='adam', activation='logistic')
model.fit(X_train, y_train.values.ravel())
print_score('MLP', y_test, model.predict(X_test))
print('mlp trained in {:.2f} seconds'.format(time() - start))

/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Naive
accuracy: 0.9235668789808917
precision: (0.9235668789808917, 0.0)
recall: (1.0, 0.0)
f1: (0.9602649006622517, 0.0)
Matthew: 0.0

Logreg
accuracy: 0.9235668789808917
precision: (0.9290322580645162, 0.5)
recall: (0.993103448275862, 0.08333333333333333)
f1: (0.9600000000000001, 0.14285714285714285)
Matthew: 0.18109070935025184

KNN
accuracy: 0.9235668789808917
precision: (0.9262820512820513, 0.5)
recall: (0.996551724137931, 0.041666666666666664)
f1: (0.9601328903654486, 0.07692307692307693)
Matthew: 0.1276393905848935

RF
accuracy: 0.8949044585987261
precision: (0.9297658862876255, 0.2)
recall: (0.9586206896551724, 0.125)
f1: (0.9439728353140917, 0.15384615384615385)
Matthew: 0.10416867525838043

GBTree
accuracy: 0.9171974522292994
precision: (0.9459459459459459, 0.4444444444444444)
recall: (0.9655172413793104, 0.3333333333333333)
f1: (0.9556313993174061, 0.380952380952381)
Matthew: 0.3415675519285482

gbt trained in 8.96 seconds
MLP
accuracy: 0.9140127388535032
precision: (0.928338

In [61]:
### from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
import numpy
# fix random seed for reproducibility
numpy.random.seed(12)


# create model
model = Sequential()
model.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(255, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model

def evaluate_model():
        y_hat = [np.round(y) for y in model.predict(X_test)]
        print_score('Keras MLP', y_test, y_hat)


class EvalCallback(Callback):
    def __init__(self, interval=5):
        super(Callback, self).__init__()
        self.interval = interval

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.interval == 0:
            evaluate_model()

start = time()
model.fit(X_train, y_train, epochs=300, batch_size=50, callbacks=[EvalCallback()])
# evaluate the model
print('model trained in {:.2f} second'.format(time() - start))


Epoch 1/300
2817/2817 [==============================] - 2s 703us/step - loss: 0.4416 - acc: 0.8928
Epoch 2/300
2817/2817 [==============================] - 1s 408us/step - loss: 0.3460 - acc: 0.8967
Epoch 3/300
2817/2817 [==============================] - 1s 406us/step - loss: 0.3093 - acc: 0.8974
Epoch 4/300
2817/2817 [==============================] - 1s 402us/step - loss: 0.2590 - acc: 0.8967
Epoch 5/300
2817/2817 [==============================] - 1s 405us/step - loss: 0.2242 - acc: 0.8978
Keras MLP
accuracy: 0.9171974522292994
precision: (0.9230769230769231, 0.0)
recall: (0.993103448275862, 0.0)
f1: (0.9568106312292359, 0.0)
Matthew: -0.023032671985243936

Epoch 6/300
2817/2817 [==============================] - 1s 405us/step - loss: 0.2340 - acc: 0.8974
Epoch 7/300
2817/2817 [==============================] - 1s 411us/step - loss: 0.2144 - acc: 0.8946
Epoch 8/300
2817/2817 [==============================] - 1s 410us/step - loss: 0.2061 - acc: 0.8960
Epoch 9/300
2817/2817 [======

/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:538: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/u/48/sarsas2/unix/miniconda3/envs/def/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


2817/2817 [==============================] - 1s 459us/step - loss: 0.2037 - acc: 0.8971
Epoch 12/300
2817/2817 [==============================] - 1s 501us/step - loss: 0.1995 - acc: 0.8974
Epoch 13/300
2817/2817 [==============================] - 1s 410us/step - loss: 0.1917 - acc: 0.8971
Epoch 14/300
2817/2817 [==============================] - 1s 402us/step - loss: 0.1911 - acc: 0.8931
Epoch 15/300
2817/2817 [==============================] - 1s 402us/step - loss: 0.1903 - acc: 0.8974
Keras MLP
accuracy: 0.9235668789808917
precision: (0.9235668789808917, 0.0)
recall: (1.0, 0.0)
f1: (0.9602649006622517, 0.0)
Matthew: 0.0

Epoch 16/300
2817/2817 [==============================] - 1s 466us/step - loss: 0.1966 - acc: 0.8974
Epoch 17/300
2817/2817 [==============================] - 1s 421us/step - loss: 0.1960 - acc: 0.8974
Epoch 18/300
2817/2817 [==============================] - 1s 449us/step - loss: 0.1870 - acc: 0.8974
Epoch 19/300
2817/2817 [==============================] - 1s 413us

2817/2817 [==============================] - 1s 475us/step - loss: 0.1351 - acc: 0.9297
Epoch 72/300
2817/2817 [==============================] - 1s 399us/step - loss: 0.1242 - acc: 0.9326
Epoch 73/300
2817/2817 [==============================] - 1s 397us/step - loss: 0.1218 - acc: 0.9336
Epoch 74/300
2817/2817 [==============================] - 1s 455us/step - loss: 0.1836 - acc: 0.9301
Epoch 75/300
2817/2817 [==============================] - 1s 433us/step - loss: 0.1493 - acc: 0.9283
Keras MLP
accuracy: 0.9267515923566879
precision: (0.9377049180327869, 0.5555555555555556)
recall: (0.9862068965517241, 0.20833333333333334)
f1: (0.9613445378151261, 0.30303030303030304)
Matthew: 0.309772506728871

Epoch 76/300
2817/2817 [==============================] - 1s 456us/step - loss: 0.1358 - acc: 0.9294
Epoch 77/300
2817/2817 [==============================] - 1s 428us/step - loss: 0.1590 - acc: 0.9258
Epoch 78/300
2817/2817 [==============================] - 1s 421us/step - loss: 0.1319 - ac

2817/2817 [==============================] - 1s 398us/step - loss: 0.0913 - acc: 0.9535
Epoch 130/300
2817/2817 [==============================] - 1s 398us/step - loss: 0.0856 - acc: 0.9546
Keras MLP
accuracy: 0.8630573248407644
precision: (0.9458483754512635, 0.24324324324324326)
recall: (0.903448275862069, 0.375)
f1: (0.924162257495591, 0.2950819672131148)
Matthew: 0.22946074872503394

Epoch 131/300
2817/2817 [==============================] - 1s 393us/step - loss: 0.0744 - acc: 0.9574
Epoch 132/300
2817/2817 [==============================] - 1s 396us/step - loss: 0.0878 - acc: 0.9546
Epoch 133/300
2817/2817 [==============================] - 1s 395us/step - loss: 0.0787 - acc: 0.9563
Epoch 134/300
2817/2817 [==============================] - 1s 395us/step - loss: 0.0833 - acc: 0.9574
Epoch 135/300
2817/2817 [==============================] - 1s 398us/step - loss: 0.0816 - acc: 0.9553
Keras MLP
accuracy: 0.8726114649681529
precision: (0.9464285714285714, 0.2647058823529412)
recall: 

2817/2817 [==============================] - 1s 450us/step - loss: 0.0527 - acc: 0.9730
Epoch 187/300
2817/2817 [==============================] - 1s 477us/step - loss: 0.0567 - acc: 0.9712
Epoch 188/300
2817/2817 [==============================] - 1s 435us/step - loss: 0.0684 - acc: 0.9677
Epoch 189/300
2817/2817 [==============================] - 1s 435us/step - loss: 0.0780 - acc: 0.9613
Epoch 190/300
2817/2817 [==============================] - 1s 437us/step - loss: 0.0581 - acc: 0.9691
Keras MLP
accuracy: 0.8694267515923567
precision: (0.9368421052631579, 0.20689655172413793)
recall: (0.9206896551724137, 0.25)
f1: (0.928695652173913, 0.22641509433962265)
Matthew: 0.15663557002196976

Epoch 191/300
2817/2817 [==============================] - 1s 480us/step - loss: 0.0553 - acc: 0.9734
Epoch 192/300
2817/2817 [==============================] - 1s 465us/step - loss: 0.0555 - acc: 0.9734
Epoch 193/300
2817/2817 [==============================] - 1s 463us/step - loss: 0.0579 - acc: 0.9

2817/2817 [==============================] - 1s 430us/step - loss: 0.0441 - acc: 0.9794
Epoch 245/300
2817/2817 [==============================] - 1s 400us/step - loss: 0.0409 - acc: 0.9808
Keras MLP
accuracy: 0.8757961783439491
precision: (0.9498207885304659, 0.2857142857142857)
recall: (0.9137931034482759, 0.4166666666666667)
f1: (0.9314586994727592, 0.3389830508474576)
Matthew: 0.2789890078281339

Epoch 246/300
2817/2817 [==============================] - 1s 407us/step - loss: 0.0405 - acc: 0.9808
Epoch 247/300
2817/2817 [==============================] - 1s 402us/step - loss: 0.0548 - acc: 0.9755
Epoch 248/300
2817/2817 [==============================] - 1s 400us/step - loss: 0.0450 - acc: 0.9783
Epoch 249/300
2817/2817 [==============================] - 1s 397us/step - loss: 0.0462 - acc: 0.9773
Epoch 250/300
2817/2817 [==============================] - 1s 397us/step - loss: 0.0504 - acc: 0.9776
Keras MLP
accuracy: 0.8821656050955414
precision: (0.9438596491228071, 0.2758620689655

In [48]:
y_hat = [np.round(y) for y in model.predict(X_test)]
print_score('Keras MLP', y_test, y_hat)

Keras MLP
accuracy: 0.8630573248407644
precision: (0.9458483754512635, 0.24324324324324326)
recall: (0.903448275862069, 0.375)
f1: (0.924162257495591, 0.2950819672131148)
Matthew: 0.22946074872503394



In [ ]:
c = model.predict(X_test)
res = model.predict_proba(X_test)
[(r[1], c) for r, c in zip(res, c)]

In [ ]:
!jt -r 

That's... not horrible. Let's check which features have the greatest effect on the thing. 

In [ ]:
#coeffs = [(coef, col) for col, coef in zip(X.columns, model.coef_[0])]

import numpy as np
coefs = [np.mean(c) for c in model.coefs_[0]]
coeffs = [(coef, col) for col, coef in zip(X.columns, coefs)] # for neural net
coeffs = sorted(coeffs, reverse=True)
for a, b in coeffs: print(a, b)


In [ ]:
#coeffs = [(coef, col) for col, coef in zip(X.columns, model.coef_[0])]
#coeffs = sorted(coeffs, reverse=True)

for a, b in coeffs[:20]: print(a, b)
    
print('...')

orig = {col:coef for coef, col in coeffs}
abss = sorted([(abs(coef), col) for coef, col in coeffs])
near_zero = sorted([(orig[col], col) for coef, col in abss[:50]], reverse=True)
for a, b in near_zero: print(a, b)


print('...')

for a, b in coeffs[-20:]: print(a, b)

Let's check the non-crime label ones specifically

In [ ]:
for a, b in coeffs: print(a, b) if 'charge_desc' not in b else ...

Try linear support vector regression next

In [ ]:
from sklearn import svm

#model = svm.LinearSVR(epsilon=0.5)
#scores = model_selection.cross_val_score(model, X, y.values.ravel(), cv=10, n_jobs=2)
#scores

Marginally better, still pretty bad. Let's try support vector regression with an RBF. In principle it has an infinite-dimensional faeture space so that should get to something reasonable at the cost of possibly overfitting.

In [ ]:
from sklearn import svm

#model = svm.SVR(epsilon=0.5)
#scores = model_selection.cross_val_score(model, X, y.values.ravel(), cv=10, n_jobs=3)
#scores

It's... not bad(?) considering we are using $\frac{1}{3}$ of the data for testing on each round of the cross validation. The resulting $R^2$ values indicate $R \approx 0.6$ which is by no means an abysmal degree of correlation. At the same time, we would expect a significantly higher degree of success based on what the task actually **is**.

Might be problem with the nature of the data, more specifically not enough data for practical learning considering the size of the (unmodified) feature space. We only have ~10x data points to the number of features.

Might want to try, say, a random forest or alternatively doing some tricks with sample generation. Might also be issues regarding colinearity, but that's a bit beyond me, to be quite hones.